GENERATE ENCODINGS

In [1]:
import cv2 
import face_recognition 
import os 
import pickle 
import re 

#Import the Images 

folderPath = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Train_Images'
imagePathList = os.listdir(folderPath)
#print(imagePathList)
imageList = []
studentIds = []

for path in imagePathList: 
    imagePath = os.path.join(folderPath, path)
    imageList.append(cv2.imread(imagePath))
    studentIds.append(re.search(r'\d+', path).group())

#print((studentIds))
    
def fingEncodings (imageList):
    encodeList = []

    for image in imageList:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(image)[0]
        encodeList.append(encode)
    
    return encodeList

print("Encoding in Progress")
encodeListKnown = fingEncodings(imageList)
encodeListKnownWithIds = [encodeListKnown, studentIds]
print("Encoding Completed")

file = open('EncodeFile.p','wb')
pickle.dump(encodeListKnownWithIds, file)
print("Pickle File Generated")



Encoding in Progress
Encoding Completed
Pickle File Generated


ADD NEW IMAGES

In [3]:
import cv2 
import face_recognition 
import os 
import pickle 
import re 


##Load Existing Data
#Load the Encoded File 

file = open('EncodeFile.p','rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
print("Old IDs" ,studentIds)
print("Pickle File Loaded")

#Import the New Images 

folderPath = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Train_New_Images'
imagePathList = os.listdir(folderPath)
print(imagePathList)
imageList = []

for path in imagePathList: 
    imagePath = os.path.join(folderPath, path)
    imageList.append(cv2.imread(imagePath))
    studentIds.append(re.search(r'\d+', path).group())

#print((studentIds))
    
def fingEncodings (imageList):
    encodeList = []

    for image in imageList:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(image)[0]
        encodeList.append(encode)
    
    return encodeList

print("Encoding in Progress")
newEncodeListKnown = fingEncodings(imageList)
encodeListKnown.extend(newEncodeListKnown)
encodeListKnownWithIds = [encodeListKnown, studentIds]
#print(encodeListKnownWithIds)
print("Encoding Update Completed")

file = open('EncodeFile.p','wb')
pickle.dump(encodeListKnownWithIds, file)
print("Pickle File Generated")
print("Updated IDs", studentIds)



Old IDs ['20070829', '20231585', '20249606']
Pickle File Loaded
['19700101_Neil Armstrong.jpg', '20211005_Emanuel Macron.jpg']
Encoding in Progress
Encoding Update Completed
Pickle File Generated
Updated IDs ['20070829', '20231585', '20249606', '19700101', '20211005']


FACE RECOGNITION 

In [16]:
import cv2 
import numpy as np 
import face_recognition
from cvzone.FPS import FPS 
from cvzone import putTextRect, cornerRect 
import time 
import pandas as pd 
import os 

fpsReader = FPS(avgCount=30)


#Load the Encoded File 

file = open('EncodeFile.p','rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
print(studentIds)
print("Pickle File Loaded")
#print(encodeListKnown)

cap = cv2.VideoCapture(0)

flag =True
start_flag = True 


while True: 

    success, img = cap.read()
    H,W,_ = img.shape
    roix1, roiy1, roix2, roiy2 = int(W*0.3), int(H*0.05), int(W-0.3*W), int(H-0.4*H)
    imgroi = img[roiy1:roiy2, roix1:roix2]
    cornerRect(img, [roix1, roiy1, roix2-roix1, roiy2-roiy1],rt=2,colorR=(0,0,0),colorC=(0,255,255))
    imgSmall = cv2.resize(imgroi, (0,0),None, 0.25, 0.25)

    faceLocations = face_recognition.face_locations(imgSmall)
    encodeFaces = face_recognition.face_encodings(imgSmall, faceLocations)
    
    if start_flag:
        start_time = time.time()
        start_flag = False  

    if faceLocations:
      
      if time.time()-start_time>1:
            for encodeFace, faceLoc in zip (encodeFaces, faceLocations): 

                matches = face_recognition.compare_faces(encodeListKnown ,encodeFace)
                faceDistance = face_recognition.face_distance(encodeListKnown, encodeFace )
                cv2.rectangle(imgroi, (faceLoc[3]*4, faceLoc[0]*4), (faceLoc[1]*4, faceLoc[2]*4), (0,0,255), 2)

                # print("Matches1", matches)
                # print("Matches2", faceDistance) 
                matchIndex = np.argmin(faceDistance)
                
                if matches[matchIndex]:
                    id = studentIds[matchIndex] 
                    #print("Mathced ID", id)
                    color = (0,210,0)
                    textr =  "AUTHORIZED"

                    if flag: 
                        #Update
                        flag=False
                        print("Reached Here")
                else: 
                    color = (0,0,220)
                    textr =  "UNAUTHORIZED"

                putTextRect(img, text=textr, pos=(int(roix1+55), roiy2+90), scale=1.5, thickness=2, colorR=color)

         
    else: 
        flag = True 
        start_flag = True 
        


    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    cv2.imshow("Face Recognition", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


['20070829', '20231585', '20249606', '19700101', '20211005']
Pickle File Loaded
Reached Here


FACE ATTENDANCE

In [18]:
import cv2 
import numpy as np 
import face_recognition
import pickle
from cvzone.FPS import FPS 
from cvzone import putTextRect, cornerRect 
import time 
import pandas as pd 
import os 
from datetime import datetime


fpsReader = FPS(avgCount=30)


#Load the Encoded File 

file = open('EncodeFile.p','rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
print(studentIds)
print("Pickle File Loaded")
#print(encodeListKnown)



# Read the root database containing student information
root_database_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Database\rootDatabase.xlsx'
root_db = pd.read_excel(root_database_path)
# Convert the 'ID' column to a list
id_list = root_db['ID'].tolist()

individual_records_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Database\Individual_Records'
daily_records_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Database\Daily_Records'

last_scan_time = {}


def create_daily_file(date):
    # Create a new Excel file for the current date if it doesn't exist
    daily_file_name = os.path.join(daily_records_path, f'{date}.xlsx')
    try:
        daily_df = pd.read_excel(daily_file_name)
    except FileNotFoundError:
        # Create a new dataframe with the required columns
        daily_df = root_db[['No.', 'ID', 'Name', 'DoB']]
        daily_df['Entry'] = 'Nill'
        daily_df['Exit'] = 'Nill'
        daily_df.to_excel(daily_file_name, index=False)
    return daily_file_name

def log_entry_exit(id, timestamp):
    #global textr,color
    # Check if the ID has been scanned within the last 60 seconds
    if id in last_scan_time and (timestamp - last_scan_time[id]).total_seconds() < 10:
        print(f"ID {id}  Try again in 10 seconds.")
        # textr =  "WAIT 10 SEC"
        # color=(0,0,255)
        return False #Handles textFlag
    # textr = 'AUTHORIZED'
    # color = (0,220,0)
    student = root_db[root_db['ID'] == id].iloc[0]
    daily_file_name = create_daily_file(timestamp.strftime('%d-%m-%Y'))
    
    # Update the daily attendance
    daily_df = pd.read_excel(daily_file_name)

    # Ensure the 'Entry' and 'Exit' columns are of type str
    daily_df['Entry'] = daily_df['Entry'].astype(str)
    daily_df['Exit'] = daily_df['Exit'].astype(str)
    
    #Check if the newly added id is present

    id_list = daily_df['ID'].tolist()
    
    if id not in id_list: 
        count = len(daily_df) + 1
        name  = root_db.loc[root_db['ID'] == id, 'Name'].values[0]
        dob   = root_db.loc[root_db['ID'] == id, 'DoB'].values[0]

        new_entry = pd.DataFrame({'No.': [count], 'ID': [id], 'Name': [name], 'DoB': [dob], 'Entry': ['Nill'], 'Exit': ['Nill']})
        daily_df = pd.concat([daily_df, new_entry], ignore_index=True)
        daily_df.to_excel(daily_file_name, index=False)



    row_index = daily_df[daily_df['ID'] == id].index[0]
    
    if daily_df.at[row_index, 'Entry'] == 'Nill':
        daily_df.at[row_index, 'Entry'] = timestamp.strftime('%H:%M:%S')
    else:
        daily_df.at[row_index, 'Exit'] = timestamp.strftime('%H:%M:%S')
    daily_df.to_excel(daily_file_name, index=False)

    # Update the individual attendance
    individual_file_name = os.path.join(individual_records_path, f'{id}.xlsx')
    try:
        individual_df = pd.read_excel(individual_file_name)
    except FileNotFoundError:
        individual_df = pd.DataFrame(columns=['Date', 'Entry', 'Exit'])

    # Ensure the 'Entry' and 'Exit' columns are of type str
    individual_df['Entry'] = individual_df['Entry'].astype(str)
    individual_df['Exit'] = individual_df['Exit'].astype(str)
    
    date_str = timestamp.strftime('%d-%m-%Y')
    if date_str in individual_df['Date'].values:
        row_index = individual_df[individual_df['Date'] == date_str].index[0]
        individual_df.at[row_index, 'Exit'] = timestamp.strftime('%H:%M:%S')
    else:
        new_entry = pd.DataFrame({'Date': [date_str], 'Entry': [timestamp.strftime('%H:%M:%S')], 'Exit': [None]})
        individual_df = pd.concat([individual_df, new_entry], ignore_index=True)
    individual_df.to_excel(individual_file_name, index=False)

    # Update the last scan time for the ID
    last_scan_time[id] = timestamp
    return True ##Added for textFlag that handles delay issue


cap = cv2.VideoCapture(0)
flag =True
start_flag = True 
textFlag = True 


while True: 

    success, img = cap.read()
    H,W,_ = img.shape
    roix1, roiy1, roix2, roiy2 = int(W*0.3), int(H*0.05), int(W-0.3*W), int(H-0.4*H)
    imgroi = img[roiy1:roiy2, roix1:roix2]
    cornerRect(img, [roix1, roiy1, roix2-roix1, roiy2-roiy1],rt=2,colorR=(0,0,0),colorC=(0,255,255))
    imgSmall = cv2.resize(imgroi, (0,0),None, 0.25, 0.25)

    faceLocations = face_recognition.face_locations(imgSmall)
    encodeFaces = face_recognition.face_encodings(imgSmall, faceLocations)
    
    if start_flag:
        start_time = time.time()
        start_flag = False  

    if faceLocations:
      
      if time.time()-start_time>1:
            for encodeFace, faceLoc in zip (encodeFaces, faceLocations): 

                matches = face_recognition.compare_faces(encodeListKnown ,encodeFace)
                faceDistance = face_recognition.face_distance(encodeListKnown, encodeFace )
                x1,y1,x2,y2 = faceLoc[3]*4, faceLoc[0]*4, faceLoc[1]*4, faceLoc[2]*4
                #cv2.rectangle(imgroi, (faceLoc[3]*4, faceLoc[0]*4), (faceLoc[1]*4, faceLoc[2]*4), (0,0,255), 2)
                cornerRect(imgroi, [x1-20,y1-20, x2-x1+20,y2-y1+20], l=40, t=4, colorC=(0,0,255), rt=0)

                # print("Matches1", matches)
                # print("Matches2", faceDistance) 
                matchIndex = np.argmin(faceDistance)
                
                if matches[matchIndex]:
                    id = studentIds[matchIndex] 
                    id = int(id) 
                    name = root_db.loc[root_db['ID'] == id, 'Name'].values[0]
                    #print("Mathced ID", id)
                    # color = (0,210,0)
                    # textr =  "AUTHORIZED"


                    if flag: 
                        #Update
                        timestamp = datetime.now()
                        textFlag=log_entry_exit(id, timestamp)
                        flag=False

                        if not textFlag: 
                             textr =  "WAIT 10 SEC"
                             color=(0,0,255)
                             textFlag = True 
                             #flag = True  #If kept then face will automatically be detected after 10sec
                        else: 
                             color = (0,210,0)
                             textr =  "AUTHORIZED"
                             print("Matched ID", id, "Mathced Name", name)

                    putTextRect(img, text=name, pos=(int(roix1+55), roiy2+100), scale=1.5, thickness=2, colorR=color,colorB=(0,0,0), border=2)

                        #print("Reached Here")
                else: 
                    color = (0,0,220)
                    textr =  "UNAUTHORIZED"
                    flag = True

                putTextRect(img, text=textr, pos=(int(roix1+55), roiy2+50), scale=1.5, thickness=2, colorR=color,colorB=(0,0,0), border=2)
                

         
    else: 
        flag = True 
        start_flag = True 
        putTextRect(img, text="SCAN YOUR FACE", pos=(int(roix1+25), roiy2+50), scale=1.5, thickness=2, colorR=(210,0,0),colorB=(0,0,0), border=2)   


    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    cv2.imshow("Face Recognition", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


['20070829', '20231585', '20249606', '19700101', '20211005']
Pickle File Loaded
Matched ID 20211005 Mathced Name Emanuel Macron
Matched ID 19700101 Mathced Name Neil Armstrong
Matched ID 20211005 Mathced Name Emanuel Macron
Matched ID 19700101 Mathced Name Neil Armstrong
ID 19700101  Try again in 10 seconds.
Matched ID 19700101 Mathced Name Neil Armstrong


DOOR LOCK WITH FACE RECOGNITION

In [1]:
import cv2 
import numpy as np 
import face_recognition
import pickle
from cvzone.FPS import FPS 
from cvzone import putTextRect, cornerRect 
from cvzone.SerialModule import SerialObject 
import time 
import pandas as pd 
import os 
from datetime import datetime
import requests


TOKEN = '7230981972:AAEFo2FxODCeX03-a0aqH33Ka8xSZwcum3I'
CHAT_ID = '5029073982'

arduino = SerialObject("COM7",digits=1)
fpsReader = FPS(avgCount=30)


#Load the Encoded File 

file = open('EncodeFile.p','rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
print(studentIds)
print("Pickle File Loaded")
#print(encodeListKnown)



# Read the root database containing student information
root_database_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Database\rootDatabase.xlsx'
root_db = pd.read_excel(root_database_path)

#Log File Info 
log_file_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Database\DoorLock_Database\Door_Log.xlsx'
log_df = pd.read_excel(log_file_path)

# Define the directory where you want to save the Excel file
save_directory = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Face Recognition\Unauthorized Log'

#Function for logging information 
def log_event(id):
        
        global log_df
        # Get the name corresponding to the ID
        name = root_db.loc[root_db['ID'] == id, 'Name'].values[0]
        
        current_time = datetime.now()
        date_str = current_time.strftime("%d-%m-%Y")
        time_str = current_time.strftime("%H:%M:%S")
    
        # Create a new row to append
        new_entry = pd.DataFrame({'No.': [len(log_df) + 1,], 'ID': [id], 'Name': [name], 'Date': [date_str], 'Time': [time_str]})
        
        # Append the new row to the log DataFrame
        log_df = pd.concat([log_df, new_entry], ignore_index=True)
        
        # Write the updated DataFrame back to the Excel file
        log_df.to_excel(log_file_path, index=False)





cap = cv2.VideoCapture(0)
flag =True
start_flag = True 
saveFlag = True
count = 0 


while True: 

    success, img = cap.read()
    H,W,_ = img.shape
    roix1, roiy1, roix2, roiy2 = int(W*0.3), int(H*0.05), int(W-0.3*W), int(H-0.4*H)
    imgroi = img[roiy1:roiy2, roix1:roix2]
    cornerRect(img, [roix1, roiy1, roix2-roix1, roiy2-roiy1],rt=2,colorR=(0,0,0),colorC=(0,255,255))
    imgSmall = cv2.resize(imgroi, (0,0),None, 0.25, 0.25)

    faceLocations = face_recognition.face_locations(imgSmall)
    encodeFaces = face_recognition.face_encodings(imgSmall, faceLocations)
    
    if start_flag:
        start_time = time.time()
        start_flag = False  

    if faceLocations:
      
      if time.time()-start_time>1:
            for encodeFace, faceLoc in zip (encodeFaces, faceLocations): 

                matches = face_recognition.compare_faces(encodeListKnown ,encodeFace)
                faceDistance = face_recognition.face_distance(encodeListKnown, encodeFace )
                x1,y1,x2,y2 = faceLoc[3]*4, faceLoc[0]*4, faceLoc[1]*4, faceLoc[2]*4
                #cv2.rectangle(imgroi, (faceLoc[3]*4, faceLoc[0]*4), (faceLoc[1]*4, faceLoc[2]*4), (0,0,255), 2)
                

                # print("Matches1", matches)
                # print("Matches2", faceDistance) 
                matchIndex = np.argmin(faceDistance)
                
                if matches[matchIndex]:
                    id = studentIds[matchIndex] 
                    id = int(id) 
                    name = root_db.loc[root_db['ID'] == id, 'Name'].values[0]
                    #print("Mathced ID", id)
                    # color = (0,210,0)
                    # textr =  "AUTHORIZED"

                    if flag: 
                        arduino.sendData([1])
                        timestamp = datetime.now()
                        log_event(id)
                        flag=False
                        textr =  "AUTHORIZED"
                        color=(0,210,0)
                        #print("Mathced Name", name)

                    putTextRect(img, text=name, pos=(int(roix1+55), roiy2+100), scale=1.5, thickness=2, colorR=color,colorB=(0,0,0), border=2)

                        #print("Reached Here")
                else:
                    arduino.sendData([2]) 
                    color = (0,0,220)
                    textr =  "UNAUTHORIZED"
                    count+=1
                    flag = True

                    if saveFlag: 
                        saveFlag = False
                        # Save the cropped face image
                        orderNo = len(os.listdir(save_directory))+1
                        img_filename = os.path.join(save_directory, f'{orderNo}.jpg')
                        imgWrite = cv2.resize(img,(int(0.5*W), int(0.5*H)))
                        cv2.imwrite(img_filename, imgWrite)
                        

                    if count==2:
                            #Send the image to telegram
                            msg=f'Unauthorized_Face {orderNo}'
                            location = img_filename
                            files = {'photo':open(location,'rb')}

                            url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto?chat_id={CHAT_ID}&caption={msg}"
                            r = requests.post(url,files=files)
                            count = 0

                putTextRect(img, text=textr, pos=(int(roix1+55), roiy2+50), scale=1.5, thickness=2, colorR=color,colorB=(0,0,0), border=2)
                cornerRect(imgroi, [x1-20,y1-20, x2-x1+20,y2-y1+20], l=40, t=4, colorC=color, rt=0)
                

         
    else: 
        flag = True 
        start_flag = True
        saveFlag = True  
        putTextRect(img, text="SCAN YOUR FACE", pos=(int(roix1+25), roiy2+50), scale=1.5, thickness=2, colorR=(210,0,0),colorB=(0,0,0), border=2)   


    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    cv2.imshow("Face Recognition", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Attempt 1 of 5 to connect...
Serial Device Connected
['20070829', '20231585', '20249606', '19700101', '20211005']
Pickle File Loaded
